In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from scipy.stats import spearmanr
import re
import arviz as az
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import cellbayesassay as cba
import pickle
import attila_utils

In [2]:
experimentl = [
    'Aβ clearance (BV2 cells)',
    'Aβ release (H4 cells)',
    'Trophic factor withdrawal (primary neurons)',
    'LPS neuroinflammation (BV2 cells)',
    'Tau phoshorylation'
              ]
experimentd = dict(zip(experimentl, np.arange(len(experimentl))))
treatmentd = {
    'TI11': 'DIM-C-pPhOCH3',
    'TI12': 'C-DIM12',
}
fpath = '../../resources/cell-based-assays/CO28151_data-summary-clean.xlsx'

In [3]:
def extract_TI_exp_data(experiment, TI, control_TI='VC', batchvars=['Batch', 'Plate'], experimentd=experimentd, fpath=fpath):
    data = pd.read_excel(fpath, experimentd[experiment])
    TI_data = data.loc[data.TI == TI]
    # ensure that all treatment TI data are from the same batch:plate
    if len(TI_data.groupby(batchvars)) != 1:
        print('treatment with multiple batches')
        return(None)
    b = data.Plate == TI_data.iloc[0].loc['Plate']
    # if there's no information on the
    if not TI_data.iloc[0].isna().loc['Batch']:
        b = b & (data.Batch == data.iloc[0].loc['Batch'])
    TI_exp_data = data.loc[b]
    TI_exp_data_control = TI_exp_data.loc[TI_exp_data.TI == control_TI].copy()
    # if there's no control for the same batch:plate, use controls from all other batch:plate combinations
    if len(TI_exp_data_control) == 0:
        TI_exp_data_control = data.loc[data.TI == control_TI].copy()
    TI_exp_data_control['conc'] = control_TI
    TI_exp_data_TI = TI_exp_data.loc[TI_exp_data.TI == TI].copy()
    TI_exp_data = pd.concat([TI_exp_data_control, TI_exp_data_TI], axis=0)
    return(TI_exp_data)

experiment = experimentl[0]
TI = list(treatmentd.keys())[0]


In [4]:
datad = dict()

experiment = experimentl[0] #'Aβ clearance (BV2 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[1] #'Aβ release (H4 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[2] #'Trophic factor withdrawal (primary neurons)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[3] #'LPS neuroinflammation (BV2 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='LPS') for TI in TI_list}

datad[experiment][TI_list[0]]

,Batch,Plate,Group,IRN,TI,conc,IL-10,IL-1β,IL-6,KC/GRO,TNF-α
126,NaN,4,H1,TI11 c1_1,TI11,c1,5.55,0.56,13553.00,11.62,1919.00
127,NaN,4,H1,TI11 c1_2,TI11,c1,7.23,0.60,16151.00,11.60,2135.00
128,NaN,4,H1,TI11 c1_3,TI11,c1,6.00,0.62,7383.00,6.53,1585.00
129,NaN,4,H1,TI11 c1_4,TI11,c1,6.24,0.31,9781.00,8.30,1740.00
130,NaN,4,H1,TI11 c1_5,TI11,c1,6.06,0.73,13001.00,8.44,1854.00
131,NaN,4,H1,TI11 c1_6,TI11,c1,6.07,0.49,14188.00,8.72,2028.00
132,NaN,4,H2,TI11 c2_2,TI11,c2,6.24,0.85,14298.00,8.23,1952.00
133,NaN,4,H2,TI11 c2_3,TI11,c2,5.86,2.00,19655.00,10.46,2038.00
134,NaN,4,H2,TI11 c2_4,TI11,c2,6.22,0.88,10032.00,6.62,1586.00
135,NaN,4,H2,TI11 c2_5,TI11,c2,6.38,0.73,11128.00,6.97,1636.00


In [5]:
%connect_info

{
  "shell_port": 51575,
  "iopub_port": 51576,
  "stdin_port": 51577,
  "control_port": 51579,
  "hb_port": 51578,
  "ip": "127.0.0.1",
  "key": "3ae4ebd3-dad1fa34dbd462416481b761",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-b5ad06a5-261d-4ccb-96c3-3c7b67a146d0.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
